# Imports

In [1]:
from scheduling_funtions import *
from constraints import *
from data import *
from model import *


# Creating Data

In [2]:
staff = create_data(staff_list)
ft_staff = filter_data(staff, ft_only_staff_mask)
midnight_staff = filter_data(staff, midnight_staff_mask)
six_month_new_staff = filter_data(staff, staff_in_first_6_months_mask)

shifts = create_data(shift_list)
midnight_shifts = filter_data(shifts, midnight_shifts_mask)
not_midnight_shifts = list_difference(shifts, midnight_shifts)
late_shifts = filter_data(shifts, late_shifts_mask)
day_shifts = filter_data(shifts, day_shifts_mask)
afternoon_shifts = filter_data(shifts, afternoon_shifts_mask)
ft_shifts = filter_data(shifts, ft_shifts_mask)
not_ft_shifts = list_difference(shifts, ft_shifts)
on_call_shifts = filter_data(shifts, on_call_shifts_mask)
not_on_call_shifts = list_difference(shifts, on_call_shifts)
after_5_shifts = late_shifts + midnight_shifts
after_930_shifts = list_difference(shifts, day_shifts)

days, first_day = create_date_range(1, 1, 2021)
fris = fridays(days, first_day)
weekdays = mondays(days, first_day) + tuesdays(days, first_day) + wednesdays(days, first_day) + thursdays(days, first_day) + fris
sats = saturdays(days, first_day)
suns = sundays(days, first_day)
weekends = sats + suns

# Creating the Model

In [3]:
model = create_model()
staff_works_shift_on_day = create_model_variables_long(model, "staff_works_shift_on_day", staff, days, shifts)
staff_works_day = create_model_variables_with_sum(model, "staff_works_day", staff_works_shift_on_day, staff, days, shifts)
staff_doesnt_work_day = not_dict(staff_works_day)
staff_works_afternoon_shift = create_model_variables_with_sum(model, "staff_works_afternoon_shift", staff_works_shift_on_day, staff, days, afternoon_shifts)
staff_doesnt_work_afternoon_shift = not_dict(staff_works_afternoon_shift)
staff_works_midnight_shift = create_model_variables_with_sum(model, "staff_works_midnight_shift", staff_works_shift_on_day, staff, days, midnight_shifts)
staff_works_on_call_shift = create_model_variables_with_sum(model, "staff_works_on_call_shift", staff_works_shift_on_day, staff, days, on_call_shifts)
staff_works_ft_shift = create_model_variables_with_sum(model, "staff_works_ft_shift", staff_works_shift_on_day, staff, days, ft_shifts)
staff_works_late_shift = create_model_variables_with_sum(model, "staff_works_late_shift", staff_works_shift_on_day, staff, days, late_shifts)
staff_works_after_5_shift = create_model_variables_with_sum(model, "staff_works_after_5_shift", staff_works_shift_on_day, staff, days, after_5_shifts)
staff_doesnt_work_after_5_shift = not_dict(staff_works_after_5_shift)
staff_works_after_930_shift = create_model_variables_with_sum(model, "staff_works_after_930_shift", staff_works_shift_on_day, staff, days, after_930_shifts)
staff_works_day_shift = create_model_variables_with_sum(model, "staff_works_after_930_shift", staff_works_shift_on_day, staff, days, day_shifts)
staff_doesnt_work_day_shift = not_dict(staff_works_day_shift)
staff_productivities = create_staff_variables(model, "staff_productivity", staff_productivity_mask, 0, 6, staff_works_shift_on_day, staff, days, shifts)

obj_int_vars, obj_int_coeffs = empty_minimize_constraints()
obj_bool_vars, obj_bool_coeffs = empty_minimize_constraints()

# Adding Constraints to the Model

In [4]:
max_days_worked_hard_max = 7
max_days_worked_soft_max = 3
max_days_worked_max_cost = MAX

min_days_off_after_midnight_hard_min = 2
min_days_off_after_midnight_soft_min = 4
min_days_off_after_midnight_min_cost = HIGH

max_midnights_in_a_row_hard_max = 2
max_midnights_in_a_row_soft_max = 1
max_midnights_in_a_row_max_cost = MAX

no_late_shift_before_time_off_hard_min = 1
no_late_shift_before_time_off_soft_min = 0
no_late_shift_before_time_off_min_cost = NONE

on_call_rules_before_hard_min = 1
on_call_rules_before_soft_min = 0
on_call_rules_before_min_cost = NONE

on_call_rules_after_hard_min = 1
on_call_rules_after_soft_min = 0
on_call_rules_after_min_cost = NONE

days_off_after_consecutive_shifts_hard_min = 2
days_off_after_consecutive_shifts_soft_min = 0
days_off_after_consecutive_shifts_min_cost = NONE

days_off_between_late_and_day_shifts_hard_min = 0
days_off_between_late_and_day_shifts_soft_min = 3
days_off_between_late_and_day_shifts_min_cost = HIGH

days_off_between_late_and_afternoon_shifts_hard_min = 0
days_off_between_late_and_afternoon_shifts_soft_min = 2
days_off_between_late_and_afternoon_shifts_min_cost = HIGH

late_shifts_in_a_row_hard_max = 7
late_shifts_in_a_row_soft_max = 3
late_shifts_in_a_row_max_cost = MAX

late_shifts_in_weeks_hard_min = 0
late_shifts_in_weeks_soft_min = 0
late_shifts_in_weeks_min_cost = NONE
late_shifts_in_weeks_hard_max = 14
late_shifts_in_weeks_soft_max = 5
late_shifts_in_weeks_max_cost = MAX

avoid_consecutive_ft_shifts_hard_max = 2
avoid_consecutive_ft_shifts_soft_max = 1
avoid_consecutive_ft_shifts_max_cost = MID

no_nightshifts_before_weekend_off_hard_min = 0
no_nightshifts_before_weekend_off_soft_min = 1
no_nightshifts_before_weekend_off_min_cost = HIGH

minimize_split_weekends_cost = MAX

equalize_weekends_cost = 4
equalize_night_shifts_cost = 4
equalize_late_shifts_cost = 4
equalize_day_shifts_cost = 2
equalize_afternoon_shifts_cost = 2
equalize_weekdays_cost = 1


In [5]:
all_shifts_taken(model,
                staff_works_shift_on_day,
                staff,
                days,
                shifts)

max_days_worked_obj = max_days_worked(model,
                staff_works_day,
                staff,
                days,
                max_days_worked_hard_max,
                max_days_worked_soft_max,
                max_days_worked_max_cost)

obj_bool_vars.extend(max_days_worked_obj[0])
obj_bool_coeffs.extend(max_days_worked_obj[1])

min_days_off_after_midnight_obj = min_days_off_after_midnight(model,
                            staff_doesnt_work_day,
                            staff_works_midnight_shift, 
                            staff, 
                            days, 
                            min_days_off_after_midnight_hard_min, 
                            min_days_off_after_midnight_soft_min, 
                            min_days_off_after_midnight_min_cost)

obj_bool_vars.extend(min_days_off_after_midnight_obj[0])
obj_bool_coeffs.extend(min_days_off_after_midnight_obj[1])

midnight_physicians(model,
                    staff_works_shift_on_day,
                    midnight_staff,
                    days,
                    not_midnight_shifts)

no_midnights_within_six_months(model,
                                staff_works_shift_on_day,
                                six_month_new_staff,
                                days,
                                midnight_shifts)

max_midnights_in_a_row_obj = max_midnights_in_a_row(model,
                        staff_works_midnight_shift,
                        staff,
                        days,
                        max_midnights_in_a_row_hard_max,
                        max_midnights_in_a_row_soft_max,
                        max_midnights_in_a_row_max_cost)

obj_bool_vars.extend(max_midnights_in_a_row_obj[0])
obj_bool_coeffs.extend(max_midnights_in_a_row_obj[1])

ft_physicians(model,
            staff_works_shift_on_day,
            ft_staff,
            days,
            not_ft_shifts)

no_late_shift_before_time_off_obj = no_late_shift_before_time_off(model,
                            staff_works_day,
                            staff_doesnt_work_after_5_shift,
                            staff,
                            no_late_shift_before_time_off_hard_min,
                            no_late_shift_before_time_off_soft_min,
                            no_late_shift_before_time_off_min_cost)

obj_bool_vars.extend(no_late_shift_before_time_off_obj[0])
obj_bool_coeffs.extend(no_late_shift_before_time_off_obj[1])
                            
on_call_rules_before_obj = on_call_rules_before(model,
                    staff_works_on_call_shift,
                    staff_doesnt_work_after_5_shift,
                    staff,
                    days,
                    on_call_rules_before_hard_min,
                    on_call_rules_before_soft_min,
                    on_call_rules_before_min_cost)

obj_bool_vars.extend(on_call_rules_before_obj[0])
obj_bool_coeffs.extend(on_call_rules_before_obj[1])

on_call_rules_after_obj = on_call_rules_after(model,
                    staff_works_on_call_shift,
                    staff_doesnt_work_day_shift,
                    staff,
                    days,
                    on_call_rules_after_hard_min,
                    on_call_rules_after_soft_min,
                    on_call_rules_after_min_cost)

obj_bool_vars.extend(on_call_rules_after_obj[0])
obj_bool_coeffs.extend(on_call_rules_after_obj[1])

days_off_after_consecutive_shifts_obj = days_off_after_consecutive_shifts(model,
                                staff_works_day,
                                staff_doesnt_work_day,
                                staff,
                                days,
                                days_off_after_consecutive_shifts_hard_min,
                                days_off_after_consecutive_shifts_soft_min,
                                days_off_after_consecutive_shifts_min_cost)

obj_bool_vars.extend(days_off_after_consecutive_shifts_obj[0])
obj_bool_coeffs.extend(days_off_after_consecutive_shifts_obj[1])

[60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240]
[60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240,

In [6]:
transitions_constraints_obj = transitions_constraints(model,
                        staff_works_shift_on_day,
                        staff,
                        days)

obj_bool_vars.extend(transitions_constraints_obj[0])
obj_bool_coeffs.extend(transitions_constraints_obj[1])

days_off_between_late_and_day_shifts_obj = days_off_between_late_and_day_shifts(model,
                                    staff_doesnt_work_day,
                                    staff_works_day_shift,
                                    staff_works_late_shift,
                                    staff,
                                    days,
                                    days_off_between_late_and_day_shifts_hard_min,
                                    days_off_between_late_and_day_shifts_soft_min,
                                    days_off_between_late_and_day_shifts_min_cost)

obj_bool_vars.extend(days_off_between_late_and_day_shifts_obj[0])
obj_bool_coeffs.extend(days_off_between_late_and_day_shifts_obj[1])

days_off_between_late_and_afternoon_shifts_obj = days_off_between_late_and_afternoon_shifts(model,
                                            staff_doesnt_work_day,
                                            staff_works_afternoon_shift,
                                            staff_works_late_shift,
                                            staff,
                                            days,
                                            days_off_between_late_and_afternoon_shifts_hard_min,
                                            days_off_between_late_and_afternoon_shifts_soft_min,
                                            days_off_between_late_and_afternoon_shifts_min_cost)

obj_bool_vars.extend(days_off_between_late_and_afternoon_shifts_obj[0])
obj_bool_coeffs.extend(days_off_between_late_and_afternoon_shifts_obj[1])

late_shifts_in_a_row_obj = late_shifts_in_a_row(model,
                    staff_works_late_shift,
                    staff,
                    days,
                    late_shifts_in_a_row_hard_max,
                    late_shifts_in_a_row_soft_max,
                    late_shifts_in_a_row_max_cost)

obj_bool_vars.extend(late_shifts_in_a_row_obj[0])
obj_bool_coeffs.extend(late_shifts_in_a_row_obj[1])

late_shifts_in_weeks_obj = late_shifts_in_weeks(model,
                        staff_works_late_shift,
                        staff,
                        days,
                        late_shifts_in_weeks_hard_min,
                        late_shifts_in_weeks_soft_min,
                        late_shifts_in_weeks_min_cost,
                        late_shifts_in_weeks_hard_max,
                        late_shifts_in_weeks_soft_max,
                        late_shifts_in_weeks_max_cost)

obj_int_vars.extend(late_shifts_in_weeks_obj[0])
obj_int_coeffs.extend(late_shifts_in_weeks_obj[1])

avoid_consecutive_ft_shifts_obj = avoid_consecutive_ft_shifts(model,
                        staff_works_ft_shift,
                        staff,
                        days,
                        avoid_consecutive_ft_shifts_hard_max,
                        avoid_consecutive_ft_shifts_soft_max,
                        avoid_consecutive_ft_shifts_max_cost)

obj_int_vars.extend(avoid_consecutive_ft_shifts_obj[0])
obj_int_coeffs.extend(avoid_consecutive_ft_shifts_obj[1])

[60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240]
[60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240,

In [7]:
equalize_weekends_obj = equalize_weekends(model,
                    staff_works_day,
                    staff,
                    weekends,
                    equalize_weekends_cost,
                    triangle_costs(len(shifts), len(weekends), len(staff)))

obj_int_vars.extend(equalize_weekends_obj[0])
obj_int_coeffs.extend(equalize_weekends_obj[1])

minimize_split_weekends_obj = minimize_split_weekends(model,
                        staff_works_day,
                        staff,
                        sats,
                        suns,
                        minimize_split_weekends_cost)

obj_int_vars.extend(minimize_split_weekends_obj[0])
obj_int_coeffs.extend(minimize_split_weekends_obj[1])

equalize_night_shifts_obj = equalize_night_shifts(model,
                        staff_works_midnight_shift,
                        staff,
                        days,
                        equalize_night_shifts_cost,
                        triangle_costs(len(midnight_shifts), len(days), len(staff)))

obj_int_vars.extend(equalize_night_shifts_obj[0])
obj_int_coeffs.extend(equalize_night_shifts_obj[1])

no_nightshifts_before_weekend_off_obj = no_nightshifts_before_weekend_off(model,
                                staff_works_day,
                                staff_doesnt_work_after_5_shift,
                                staff,
                                fris,
                                sats,
                                no_nightshifts_before_weekend_off_hard_min,
                                no_nightshifts_before_weekend_off_soft_min,
                                no_nightshifts_before_weekend_off_min_cost)

obj_bool_vars.extend(no_nightshifts_before_weekend_off_obj[0])
obj_bool_coeffs.extend(no_nightshifts_before_weekend_off_obj[1])

equalize_late_shifts_obj = equalize_late_shifts(model,
                    staff_works_late_shift,
                    staff,
                    days,
                    equalize_late_shifts_cost,
                    triangle_costs(len(late_shifts), len(days), len(staff)))

obj_int_vars.extend(equalize_late_shifts_obj[0])
obj_int_coeffs.extend(equalize_late_shifts_obj[1])

equalize_day_shifts_obj = equalize_day_shifts(model,
                    staff_works_day_shift,
                    staff,
                    days,
                    equalize_day_shifts_cost,
                    triangle_costs(len(day_shifts), len(days), len(staff)))

obj_int_vars.extend(equalize_day_shifts_obj[0])
obj_int_coeffs.extend(equalize_day_shifts_obj[1])

equalize_afternoon_shifts_obj = equalize_afternoon_shifts(model,
                            staff_works_afternoon_shift,
                            staff,
                            days,
                            equalize_afternoon_shifts_cost,
                            triangle_costs(len(afternoon_shifts), len(days), len(staff)))

obj_int_vars.extend(equalize_afternoon_shifts_obj[0])
obj_int_coeffs.extend(equalize_afternoon_shifts_obj[1])

equalize_weekdays_obj = equalize_weekdays(model,
                    staff_works_day,
                    staff,
                    weekdays,
                    equalize_weekdays_cost,
                    triangle_costs(len(shifts), len(weekdays), len(staff)))

obj_int_vars.extend(equalize_weekdays_obj[0])
obj_int_coeffs.extend(equalize_weekdays_obj[1])

In [8]:
# apply_requests(staff_works_shift_on_day,
#                 staff_works_day,
#                 days,
#                 obj_int_vars,
#                 obj_int_coeffs,
#                 requests)

# apply_productivity(model,
#                     staff_productivities,
#                     staff,
#                     days,
#                     shifts,
#                     3,
#                     obj_int_vars,
#                     obj_int_coeffs)

# Solving the Model

In [9]:
# All constraints together takes 1.5 hours to find a feasible solution
import time

start = time.time()

model.Minimize(
        sum(obj_bool_vars[i] * obj_bool_coeffs[i]
            for i in range(len(obj_bool_vars))) +
            sum(obj_int_vars[i] * obj_int_coeffs[i]
            for i in range(len(obj_int_vars))))

solver = create_solver(120)
solution_printer = VarArraySolutionPrinter(staff_works_shift_on_day)
status = solver.Solve(model)

print(solver.StatusName(status))

if status == FEASIBLE or status == OPTIMAL:

    print('Maximum of objective function: %i' % solver.ObjectiveValue())
    print()
    for d in days:
        print('Day', d)
        for s in shifts:
            for m in staff:
                if solver.Value(staff_works_shift_on_day[(m, d, s)]) == 1:
                    print(staff_list[m], 'works shift', shift_list[s])
        print()

end = time.time()
print(f"time elapsed: {end - start}")

FEASIBLE
Maximum of objective function: 5396

Day 0
Nora works shift 0700 - 1500
Charlotte works shift 0730 - 1530 (FT)
Victoria works shift 0930 - 1730
Aria works shift 1200 - 2000
Abigail works shift 1400 - 2200
Grace works shift 1530 - 2330 (FT)
Penelope works shift 1600 - 2400
Emily works shift 1800 - 0200
Sophia works shift 2000 - 0400
Scarlett works shift 2200 - 0400
Elizabeth works shift 2359 - 0700
Amelia works shift On Call

Day 1
Luna works shift 0700 - 1500
Layla works shift 0730 - 1530 (FT)
Madison works shift 0930 - 1730
Chloe works shift 1200 - 2000
Eleanor works shift 1400 - 2200
Victoria works shift 1530 - 2330 (FT)
Avery works shift 1600 - 2400
Evelyn works shift 1800 - 0200
Sofia works shift 2000 - 0400
Mia works shift 2200 - 0400
Harper works shift 2359 - 0700
Ella works shift On Call

Day 2
Sophia works shift 0700 - 1500
Avery works shift 0730 - 1530 (FT)
Grace works shift 0930 - 1730
Madison works shift 1200 - 2000
Camila works shift 1400 - 2200
Penelope works shif

# Tests

In [10]:
staff_works_day_results = {}
staff_works_midnight_shift_results = {}
staff_works_late_shift_results = {}
staff_works_day_shift_results = {}
staff_works_shift_on_day_results = {}
staff_works_ft_shift_results = {}
staff_works_after_5_shift_results = {}
staff_works_on_call_shift_results = {}
staff_works_after_930_shift_results = {}
staff_works_afternoon_shift_results = {}
for m in staff:
    staff_works_day_results[m] = [solver.Value(staff_works_day[m, d]) for d in days]
    staff_works_midnight_shift_results[m] = [solver.Value(staff_works_midnight_shift[m, d]) for d in days]
    staff_works_late_shift_results[m] = [solver.Value(staff_works_late_shift[m, d]) for d in days]
    staff_works_day_shift_results[m] = [solver.Value(staff_works_day_shift[m, d]) for d in days]
    staff_works_ft_shift_results[m] = [solver.Value(staff_works_ft_shift[m, d]) for d in days]
    staff_works_after_5_shift_results[m] = [solver.Value(staff_works_after_5_shift[m,d]) for d in days]
    staff_works_on_call_shift_results[m] = [solver.Value(staff_works_on_call_shift[m,d]) for d in days]
    staff_works_after_930_shift_results[m] = [solver.Value(staff_works_after_930_shift[m,d]) for d in days]
    staff_works_afternoon_shift_results[m] = [solver.Value(staff_works_afternoon_shift[m,d]) for d in days]
    for d in days:
        staff_works_shift_on_day_results[m, d] = [solver.Value(staff_works_shift_on_day[(m, d, s)]) for s in shifts]

## Hard Constraints

In [11]:
from test import *

all_shifts_taken_test(staff_works_shift_on_day_results, staff, days, shifts)
  
no_two_shifts_on_same_day_test(staff_works_shift_on_day_results)

max_days_worked_test(staff_works_day_results,
                    max_days_worked_hard_max,
                    max_days_worked_soft_max, 
                    obj_result(solver, max_days_worked_obj))

min_days_off_after_midnight_test(staff_works_day_results,
                                 min_days_off_after_midnight_hard_min, 
                                 min_days_off_after_midnight_soft_min,
                                 obj_result(solver, min_days_off_after_midnight_obj),
                                 Prior(staff_works_midnight_shift_results, [1], True))

midnight_physicians_test(staff_works_shift_on_day_results, shifts)

no_midnights_within_six_months_test(staff_works_midnight_shift_results, shifts)

max_midnights_in_a_row_test(staff_works_midnight_shift_results, 
                            max_midnights_in_a_row_hard_max,
                            max_midnights_in_a_row_soft_max,
                            obj_result(solver, max_midnights_in_a_row_obj))

ft_physicians_test(staff_works_shift_on_day_results, shifts)

no_late_shift_before_time_off_test(staff_works_day_results, staff_works_after_5_shift_results)

# # Physicians can work the 0930 shifts or earlier prior to working on call. They can work starting no earlier than 11 the day after on call.
# # On call shift - day after rules
#     for key, results in staff_works_on_call_shift_results.items():
#         for idx in results:
#             if idx > 0 and results[idx]:
#                 assert(staff_works_after_930_shift_results[key][idx-1] == 0)

#     for key, result in staff_works_on_call_shift_results.items():
#         for idx in result:
#             if idx < len(days)-1 and result[idx]:
#                 assert(staff_works_day_shift_results[key][idx+1] == 0)

# # 2 days off after 3 to 7 days of work in a row
# for key, result in staff_works_day_results.items():
#     assert(not detect_pattern(result, "11101"))

print("ALL HARD CONSTRAINTS SATISFIED")

Max days worked consecutively
Objective function: 0
	3 from range 3 to 7: 11

	4 from range 3 to 7: 0

	5 from range 3 to 7: 0

	6 from range 3 to 7: 0

	7 from range 3 to 7: 0

Min days off after midnight shift
Objective function: 400
	2 from range 2 to 4: 0

	3 from range 2 to 4: 10

	4 from range 2 to 4: 9

Max midnights worked consecutively
Objective function: 180
	1 from range 1 to 2: 29

	2 from range 1 to 2: 3

ALL HARD CONSTRAINTS SATISFIED


## Pie in the sky

In [34]:
print(HIGH)

40


In [13]:
# #Ensure requests work as intended
# number_of_requests_respected = 0
# number_of_requests_disrespected = 0

# for staff2, day2, shift2, coef in requests:
#     if shift2 == -1 and coef > 0:
#         if solver.Value(staff_works_day_results[staff2, day2]):
#             number_of_requests_disrespected += 1
#         else:
#             number_of_requests_respected += 1
#     elif shift2 == -1 and coef < 0:
#         if solver.Value(staff_works_day_results[staff2, day2]):
#             number_of_requests_respected += 1
#         else:
#             number_of_requests_disrespected += 1
#     elif coef > 0:
#         if solver.Value(staff_works_shift_on_day_results[staff2, day2, shift2]):
#             number_of_requests_disrespected += 1
#         else:
#             number_of_requests_respected += 1
#     elif coef < 0:
#         if solver.Value(staff_works_shift_on_day_results[staff2, day2, shift2]):
#             number_of_requests_respected += 1
#         else:
#             number_of_requests_disrespected += 1

# print(f"number of requests respected: {number_of_requests_respected}")
# print(f"number of requests disrespected: {number_of_requests_disrespected}")

In [14]:
# #Ensure requests work as intended
# productivity_under = 0
# productivity_over = 0

# for d in days:
#     for wind in window(shifts[:-1], 3):
#         prod = [solver.Value(productivities[m,d,s])
#         for m in staff 
#         for s in wind]
#         sums = sum(prod)
#         if sums < 6:
#             productivity_under += 1
#         if sums > 15:
#             productivity_over += 1

# print(f"number of times theres too little productivity: {productivity_under}")
# print(f"number of times theres too much productivity: {productivity_over}")

## Soft Constraints

In [15]:
# General principle avoid shift times changing too much day to day
# Shifts should have same start time to 2.5 hours later compared to previous shift (the 2 hours later can be relaxed to 3,4 perhaps)
# No shifts that start more than 1.5 hours earlier than the shift on the previous day
penalized_transitions = []

for shift in list(permutations(shift_list[:-1], 2)):
    t1 = float(shift[0][0:2] + '.' + shift[0][2:4])
    t2 = float(shift[1][0:2] + '.' + shift[1][2:4])
    if t2 - t1 > 2.5:
        penalized_transitions.append(((shift_list.index(shift[0]), shift_list.index(shift[1])), 1))
    elif t2 - t1 < -1.5:
        penalized_transitions.append(((shift_list.index(shift[0]), shift_list.index(shift[1])), 3))


transitions = {elem[0]: elem[1] for elem in penalized_transitions}
jumps_forward_25 = 0
jumps_backwards_15 = 0
previous_shift = -1
for key, result in staff_works_shift_on_day_results.items():
    if previous_shift == -1:
        try:
            previous_shift = result.index(1)
        except:
            previous_shift = -1
    else:
        try:
            ans = transitions[(previous_shift, result.index(1))]
            if ans == 1:
                jumps_forward_25 += 1
            elif ans == 3:
                jumps_backwards_15 += 1
            previous_shift = result.index(1)
        except:
            previous_shift = -1

    if key[1] == len(days) - 1:
        previous_shift = -1 

print(f"Number of times shifts jump forwards 2.5 hours: {jumps_forward_25}")
print(f"Number of times shifts jump backwards 1.5 hours: {jumps_forward_25}")
print()

Number of times shifts jump forwards 2.5 hours: 16
Number of times shifts jump backwards 1.5 hours: 16



In [16]:
# 3 days off after midnight. 4 even better
two_days_after_midnight = 0
three_days_after_midnight = 0
four_days_after_midnight = 0
five_days_after_midnight = 0
for key in staff_works_midnight_shift_results.keys() and staff_works_day_results.keys():
    for idx in days[:-4]:
        if staff_works_midnight_shift_results[key][idx] == 1 and staff_works_midnight_shift_results[key][idx+1] == 0:
            if sum(staff_works_day_results[key][idx+1:idx+3]) == 0 and sum(staff_works_day_results[key][idx+1:idx+4]) == 1:
                two_days_after_midnight += 1
    for idx in days[:-5]:
        if staff_works_midnight_shift_results[key][idx] == 1 and staff_works_midnight_shift_results[key][idx+1] == 0:
            if sum(staff_works_day_results[key][idx+1:idx+4]) == 0 and sum(staff_works_day_results[key][idx+1:idx+5]) == 1:
                three_days_after_midnight += 1
    for idx in days[:-6]:
        if staff_works_midnight_shift_results[key][idx] == 1 and staff_works_midnight_shift_results[key][idx+1] == 0:
            if sum(staff_works_day_results[key][idx+1:idx+5]) == 0 and sum(staff_works_day_results[key][idx+1:idx+6]) == 1:
                four_days_after_midnight += 1
    for idx in days[:-6]:
        if staff_works_midnight_shift_results[key][idx] == 1 and staff_works_midnight_shift_results[key][idx+1] == 0:
            if sum(staff_works_day_results[key][idx+1:idx+6]) == 0:
                five_days_after_midnight += 1
print(f"Number of times an employee gets 2 days off after midnights: {two_days_after_midnight}")
print(f"Number of times an employee gets 3 days off after midnights: {three_days_after_midnight}")
print(f"Number of times an employee gets 4 days off after midnights: {four_days_after_midnight}")
print(f"Number of times an employee gets 5 days off after midnights: {five_days_after_midnight}")
print()

Number of times an employee gets 2 days off after midnights: 0
Number of times an employee gets 3 days off after midnights: 9
Number of times an employee gets 4 days off after midnights: 9
Number of times an employee gets 5 days off after midnights: 6



In [17]:
#Max 1 midnight in a row
two_midnights_in_a_row = 0
for key, result in staff_works_midnight_shift_results.items():
    two_midnights_in_a_row += detect_pattern_soft(result, '11')
print(f"Number of times an employee works 2 midnights in a row: {two_days_after_midnight}")
print()

Number of times an employee works 2 midnights in a row: 0



In [18]:
# 3 days off when transitioning from late shift to day shift 
one_days_off_from_late_to_day = 0
two_days_off_from_late_to_day = 0
three_days_off_from_late_to_day = 0
for key in staff_works_day_shift_results.keys() and staff_works_late_shift_results.keys():
    for idx, elem in enumerate(staff_works_late_shift_results[key][:-2]):
        if elem == 1 and staff_works_day_shift_results[key][idx + 2]:
            if sum(staff_works_day_shift_results[key][idx+1:idx+2]) == 0:
                one_days_off_from_late_to_day += 1
    for idx, elem in enumerate(staff_works_late_shift_results[key][:-3]):
        if elem == 1 and staff_works_day_shift_results[key][idx + 3]:
            if sum(staff_works_day_shift_results[key][idx+1:idx+3]) == 0:
                two_days_off_from_late_to_day += 1
    for idx, elem in enumerate(staff_works_late_shift_results[key][:-4]):
        if elem == 1 and staff_works_day_shift_results[key][idx + 4]:
            if sum(staff_works_day_shift_results[key][idx+1:idx+4]) == 0:
                three_days_off_from_late_to_day += 1
print(f"Number of times there is a day shift within 1 days of a late shift: {one_days_off_from_late_to_day}")
print(f"Number of times there is a day shift within 2 days of a late shift: {two_days_off_from_late_to_day}")
print(f"Number of times there is a day shift within 3 days of a late shift: {three_days_off_from_late_to_day}")
print()

Number of times there is a day shift within 1 days of a late shift: 9
Number of times there is a day shift within 2 days of a late shift: 1
Number of times there is a day shift within 3 days of a late shift: 8



In [19]:
# 2 days off when transitioning from late shift to afternoon shift (although this transition should be avoided) 
one_days_off_from_late_to_afternoon = 0
two_days_off_from_late_to_afternoon = 0
for key in staff_works_afternoon_shift_results.keys() and staff_works_late_shift_results.keys():
    for idx, elem in enumerate(staff_works_late_shift_results[key][:-2]):
        if elem == 1 and staff_works_afternoon_shift_results[key][idx + 2]:
            if sum(staff_works_afternoon_shift_results[key][idx+1:idx+2]) == 0:
                one_days_off_from_late_to_afternoon += 1
    for idx, elem in enumerate(staff_works_late_shift_results[key][:-3]):
        if elem == 1 and staff_works_afternoon_shift_results[key][idx + 3]:
            if sum(staff_works_afternoon_shift_results[key][idx+1:idx+3]) == 0:
                two_days_off_from_late_to_afternoon += 1
print(f"Number of times there is a afternoon shift within 1 days of a late shift: {one_days_off_from_late_to_afternoon}")
print(f"Number of times there is a afternoon shift within 2 days of a late shift: {two_days_off_from_late_to_afternoon}")
print()

Number of times there is a afternoon shift within 1 days of a late shift: 6
Number of times there is a afternoon shift within 2 days of a late shift: 15



In [20]:
# 3 late shifts in a row maximum - late shifts are 1800, 2000, 2200. The ability to set what a late shift is would be great
three_days_in_a_row_late = 0
four_days_in_a_row_late = 0
five_days_in_a_row_late = 0
for key, result in staff_works_late_shift_results.items():
    three_days_in_a_row_late += detect_pattern_soft(result, '111')
    four_days_in_a_row_late += detect_pattern_soft(result, '1111')
    five_days_in_a_row_late += detect_pattern_soft(result, '11111')
print(f"Number of times an employee works 3 late days in a row: {three_days_in_a_row_late}")
print(f"Number of times an employee works 4 late days in a row: {four_days_in_a_row_late}")
print(f"Number of times an employee works 5 late days in a row: {five_days_in_a_row_late}")
print()

Number of times an employee works 3 late days in a row: 0
Number of times an employee works 4 late days in a row: 0
Number of times an employee works 5 late days in a row: 0



In [21]:
# 5 late shifts in two weeks maximum


In [22]:
# Work maximum of 5 days in a row
#Max 1 midnight in a row
five_days_in_a_row = 0
six_days_in_a_row = 0
seven_days_in_a_row = 0
for key, result in staff_works_day_results.items():
    five_days_in_a_row += detect_pattern_soft(result, '11111')
    six_days_in_a_row += detect_pattern_soft(result, '111111')
    seven_days_in_a_row += detect_pattern_soft(result, '1111111')
print(f"Number of times an employee works 5 days in a row: {five_days_in_a_row}")
print(f"Number of times an employee works 6 days in a row: {six_days_in_a_row}")
print(f"Number of times an employee works 7 days in a row: {seven_days_in_a_row}")
print()

Number of times an employee works 5 days in a row: 0
Number of times an employee works 6 days in a row: 0
Number of times an employee works 7 days in a row: 0



In [23]:
# Avoid FT shifts (0730,1530) on consecutive days
two_ft_days_in_a_row = 0
for key, result in staff_works_ft_shift_results.items():
    if ft_only_staff_mask[key]:
        continue
    two_ft_days_in_a_row += detect_pattern_soft(result, '11')
print(f"Number of times an employee works 2 ft shifts in a row: {two_ft_days_in_a_row}")
print()

Number of times an employee works 2 ft shifts in a row: 0



In [24]:
print("SOFT SUMMARY")
print()
print(f"Number of times shifts jump forwards 2.5 hours: {jumps_forward_25}")
print(f"Number of times shifts jump backwards 1.5 hours: {jumps_forward_25}")
print()
print(f"Number of times an employee gets 2 days off after midnights: {two_days_after_midnight}")
print(f"Number of times an employee gets 3 days off after midnights: {three_days_after_midnight}")
print(f"Number of times an employee gets 4 days off after midnights: {four_days_after_midnight}")
print(f"Number of times an employee gets 5 days off after midnights: {five_days_after_midnight}")
print()
print(f"Number of times an employee works 2 midnights in a row: {two_days_after_midnight}")
print()
print(f"Number of times there is a day shift within 1 days of a late shift: {one_days_off_from_late_to_day}")
print(f"Number of times there is a day shift within 2 days of a late shift: {two_days_off_from_late_to_day}")
print(f"Number of times there is a day shift within 3 days of a late shift: {three_days_off_from_late_to_day}")
print()
print(f"Number of times there is a afternoon shift within 1 days of a late shift: {one_days_off_from_late_to_afternoon}")
print(f"Number of times there is a afternoon shift within 2 days of a late shift: {two_days_off_from_late_to_afternoon}")
print()
print(f"Number of times an employee works 3 late days in a row: {three_days_in_a_row_late}")
print(f"Number of times an employee works 4 late days in a row: {four_days_in_a_row_late}")
print(f"Number of times an employee works 5 late days in a row: {five_days_in_a_row_late}")
print()
print(f"Number of times an employee works 5 days in a row: {five_days_in_a_row}")
print(f"Number of times an employee works 6 days in a row: {six_days_in_a_row}")
print(f"Number of times an employee works 7 days in a row: {seven_days_in_a_row}")
print()
print(f"Number of times an employee works 2 ft shifts in a row: {two_ft_days_in_a_row}")
print()

SOFT SUMMARY

Number of times shifts jump forwards 2.5 hours: 16
Number of times shifts jump backwards 1.5 hours: 16

Number of times an employee gets 2 days off after midnights: 0
Number of times an employee gets 3 days off after midnights: 9
Number of times an employee gets 4 days off after midnights: 9
Number of times an employee gets 5 days off after midnights: 6

Number of times an employee works 2 midnights in a row: 0

Number of times there is a day shift within 1 days of a late shift: 9
Number of times there is a day shift within 2 days of a late shift: 1
Number of times there is a day shift within 3 days of a late shift: 8

Number of times there is a afternoon shift within 1 days of a late shift: 6
Number of times there is a afternoon shift within 2 days of a late shift: 15

Number of times an employee works 3 late days in a row: 0
Number of times an employee works 4 late days in a row: 0
Number of times an employee works 5 late days in a row: 0

Number of times an employee wo

## Goals

In [25]:
weekend_distance = 0
for m in staff:
    value = sum([solver.Value(staff_works_day[(m, d)]) for d in weekends])
    print(f"{staff_list[m]} works {value} weekends")
    weekend_distance += abs(triangle_costs(len(shifts), len(weekends), len(staff)) - value)

print(f"Distance days from target weekends: {weekend_distance}")

Olivia works 4 weekends
Emma works 4 weekends
Ava works 4 weekends
Charlotte works 4 weekends
Sophia works 4 weekends
Amelia works 4 weekends
Isabella works 4 weekends
Mia works 4 weekends
Evelyn works 4 weekends
Harper works 4 weekends
Camila works 4 weekends
Gianna works 4 weekends
Abigail works 4 weekends
Luna works 4 weekends
Ella works 4 weekends
Elizabeth works 4 weekends
Sofia works 4 weekends
Emily works 4 weekends
Avery works 4 weekends
Mila works 4 weekends
Aria works 4 weekends
Scarlett works 4 weekends
Penelope works 4 weekends
Layla works 4 weekends
Chloe works 4 weekends
Victoria works 4 weekends
Madison works 4 weekends
Eleanor works 4 weekends
Grace works 4 weekends
Nora works 4 weekends
Distance days from target weekends: 0


In [26]:
split_weekends = 0
for m in staff:
    sat = [solver.Value(staff_works_day[m, d]) for d in sats]
    sun = [solver.Value(staff_works_day[m, d]) for d in suns]
    for d in sats:
        if d < len(suns) and sat[d] and sun[d]:
            split_weekends += 1

print(f"Number of split weekends: {split_weekends}")

Number of split weekends: 7


In [27]:
night_distance = 0
for m in staff:
    value = sum([solver.Value(staff_works_midnight_shift[(m, d)]) for d in days])
    print(f"{staff_list[m]} works {value} nights")
    night_distance += abs(triangle_costs(len(midnight_shifts), len(days), len(staff)) - value)

print(f"Distance days from target night shifts: {night_distance}")

Olivia works 2 nights
Emma works 2 nights
Ava works 0 nights
Charlotte works 0 nights
Sophia works 1 nights
Amelia works 1 nights
Isabella works 0 nights
Mia works 2 nights
Evelyn works 2 nights
Harper works 2 nights
Camila works 2 nights
Gianna works 2 nights
Abigail works 1 nights
Luna works 2 nights
Ella works 1 nights
Elizabeth works 2 nights
Sofia works 0 nights
Emily works 0 nights
Avery works 0 nights
Mila works 1 nights
Aria works 2 nights
Scarlett works 0 nights
Penelope works 2 nights
Layla works 2 nights
Chloe works 1 nights
Victoria works 0 nights
Madison works 0 nights
Eleanor works 0 nights
Grace works 1 nights
Nora works 0 nights
Distance days from target night shifts: 29


In [28]:
# weekends_after_working_fri = 0
# for m in staff:
#     for d in sats:
#         sat = solver.Value(staff_works_day[m, d])
#         if d and sat:
#             fri = sum([solver.Value(staff_works_late_shift_results[m, d-1])] + [solver.Value(staff_works_midnight_shift[m, d-1])])
#             if fri:
#                 weekends_after_working_fri += 1

# print(f"Number of weekends after working past 5 on friday: {weekends_after_working_fri}")

In [29]:
late_distance = 0
for m in staff:
    value = sum([solver.Value(staff_works_late_shift[(m, d)]) for d in days])
    print(f"{staff_list[m]} works {value} lates")
    late_distance += abs(triangle_costs(len(late_shifts), len(days), len(staff)) - value)

print(f"Distance days from target late shifts: {late_distance}")

Olivia works 1 lates
Emma works 4 lates
Ava works 3 lates
Charlotte works 3 lates
Sophia works 4 lates
Amelia works 2 lates
Isabella works 3 lates
Mia works 2 lates
Evelyn works 3 lates
Harper works 0 lates
Camila works 1 lates
Gianna works 4 lates
Abigail works 2 lates
Luna works 3 lates
Ella works 4 lates
Elizabeth works 4 lates
Sofia works 4 lates
Emily works 4 lates
Avery works 4 lates
Mila works 3 lates
Aria works 4 lates
Scarlett works 3 lates
Penelope works 2 lates
Layla works 4 lates
Chloe works 4 lates
Victoria works 3 lates
Madison works 3 lates
Eleanor works 4 lates
Grace works 4 lates
Nora works 4 lates
Distance days from target late shifts: 27


In [30]:
day_distance = 0
for m in staff:
    value = sum([solver.Value(staff_works_day_shift[(m, d)]) for d in days])
    print(f"{staff_list[m]} works {value} days")
    day_distance += abs(triangle_costs(len(day_shifts), len(days), len(staff)) - value)

print(f"Distance days from target day shifts: {day_distance}")

Olivia works 4 days
Emma works 2 days
Ava works 3 days
Charlotte works 4 days
Sophia works 3 days
Amelia works 2 days
Isabella works 2 days
Mia works 4 days
Evelyn works 3 days
Harper works 4 days
Camila works 2 days
Gianna works 3 days
Abigail works 4 days
Luna works 2 days
Ella works 2 days
Elizabeth works 4 days
Sofia works 3 days
Emily works 3 days
Avery works 4 days
Mila works 2 days
Aria works 2 days
Scarlett works 4 days
Penelope works 4 days
Layla works 3 days
Chloe works 3 days
Victoria works 4 days
Madison works 4 days
Eleanor works 3 days
Grace works 2 days
Nora works 4 days
Distance days from target day shifts: 27


In [31]:
afternoon_distance = 0
for m in staff:
    value = sum([solver.Value(staff_works_afternoon_shift[(m, d)]) for d in days])
    print(f"{staff_list[m]} works {value} afternoons")
    afternoon_distance += abs(triangle_costs(len(afternoon_shifts), len(days), len(staff)) - value)

print(f"Distance days from target afternoon shifts: {afternoon_distance}")

Olivia works 3 afternoons
Emma works 4 afternoons
Ava works 5 afternoons
Charlotte works 4 afternoons
Sophia works 5 afternoons
Amelia works 5 afternoons
Isabella works 5 afternoons
Mia works 5 afternoons
Evelyn works 5 afternoons
Harper works 4 afternoons
Camila works 5 afternoons
Gianna works 2 afternoons
Abigail works 3 afternoons
Luna works 4 afternoons
Ella works 2 afternoons
Elizabeth works 2 afternoons
Sofia works 4 afternoons
Emily works 5 afternoons
Avery works 4 afternoons
Mila works 5 afternoons
Aria works 4 afternoons
Scarlett works 5 afternoons
Penelope works 5 afternoons
Layla works 2 afternoons
Chloe works 4 afternoons
Victoria works 5 afternoons
Madison works 5 afternoons
Eleanor works 5 afternoons
Grace works 5 afternoons
Nora works 3 afternoons
Distance days from target afternoon shifts: 26


In [32]:
weekday_distance = 0
for m in staff:
    value = sum([solver.Value(staff_works_day[(m, d)]) for d in weekdays])
    print(f"{staff_list[m]} works {value} weekdays")
    weekday_distance += abs(triangle_costs(len(shifts), len(weekdays), len(staff)) - value)

print(f"Distance days from target weekdays: {weekday_distance}")

Olivia works 7 weekdays
Emma works 8 weekdays
Ava works 9 weekdays
Charlotte works 9 weekdays
Sophia works 9 weekdays
Amelia works 7 weekdays
Isabella works 8 weekdays
Mia works 9 weekdays
Evelyn works 9 weekdays
Harper works 6 weekdays
Camila works 9 weekdays
Gianna works 9 weekdays
Abigail works 7 weekdays
Luna works 7 weekdays
Ella works 8 weekdays
Elizabeth works 9 weekdays
Sofia works 9 weekdays
Emily works 9 weekdays
Avery works 9 weekdays
Mila works 7 weekdays
Aria works 9 weekdays
Scarlett works 9 weekdays
Penelope works 9 weekdays
Layla works 8 weekdays
Chloe works 9 weekdays
Victoria works 9 weekdays
Madison works 9 weekdays
Eleanor works 9 weekdays
Grace works 9 weekdays
Nora works 8 weekdays
Distance days from target weekdays: 18


In [33]:
print("GOAL SUMMARY")
print()
print(f"Distance days from target weekends: {weekend_distance}")
print()
print(f"Number of split weekends: {split_weekends}")
print()
print(f"Distance days from target night shifts: {night_distance}")
print()
print(f"Number of weekends after working past 5 on friday: {weekends_after_working_fri}")
print()
print(f"Distance days from target late shifts: {late_distance}")
print()
print(f"Distance days from target day shifts: {day_distance}")
print()
print(f"Distance days from target afternoon shifts: {afternoon_distance}")
print()
print(f"Distance days from target weekdays: {weekday_distance}")
print()

GOAL SUMMARY

Distance days from target weekends: 0

Number of split weekends: 7

Distance days from target night shifts: 29



NameError: name 'weekends_after_working_fri' is not defined

In [51]:
# new_works = {k:v for (k,v) in works.items() if k[1] >= 0}
# save_shifts_to_file(works, len(staff_list), jan[1], len(shift_list), solver)

NameError: name 'works' is not defined

In [52]:
print("Everything passed")

Everything passed
